# [모델 매개변수 최적화하기](https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork()

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [7]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306197  [    0/60000]
loss: 2.295336  [ 6400/60000]
loss: 2.284924  [12800/60000]
loss: 2.281925  [19200/60000]
loss: 2.264157  [25600/60000]
loss: 2.264079  [32000/60000]
loss: 2.245133  [38400/60000]
loss: 2.236146  [44800/60000]
loss: 2.226029  [51200/60000]
loss: 2.222194  [57600/60000]
Test Error: 
 Accuracy: 46.9%, Avg loss: 2.218874 

Epoch 2
-------------------------------
loss: 2.209860  [    0/60000]
loss: 2.204240  [ 6400/60000]
loss: 2.183577  [12800/60000]
loss: 2.205658  [19200/60000]
loss: 2.143739  [25600/60000]
loss: 2.164968  [32000/60000]
loss: 2.130480  [38400/60000]
loss: 2.111313  [44800/60000]
loss: 2.111465  [51200/60000]
loss: 2.098246  [57600/60000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 2.096189 

Epoch 3
-------------------------------
loss: 2.081537  [    0/60000]
loss: 2.072093  [ 6400/60000]
loss: 2.032490  [12800/60000]
loss: 2.085639  [19200/60000]
loss: 1.965762  [25600/60000]
loss: 2.016902  [32000/600